In [3]:
!pip install catboost lightgbm

In [29]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [30]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [31]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [32]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 12, iterations = 1050)

In [33]:
target = df["target"]

In [34]:
df_new = df_new.fillna(0)

In [35]:
df_new['DepTime'] = df_new['DepTime'] / 100

In [36]:
df_new['ArrTime'] = df_new['ArrTime'] / 100

In [37]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [38]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [39]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [40]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [41]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,13.34,14.40,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,17.06,20.16,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,13.23,15.07,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,20.39,22.38,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,16.28,17.44,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [42]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.252955
0:	learn: 0.6216800	total: 12.8s	remaining: 3h 43m 57s
1:	learn: 0.5781440	total: 18.8s	remaining: 2h 44m 25s
2:	learn: 0.5510796	total: 25.6s	remaining: 2h 28m 51s
3:	learn: 0.5345742	total: 31.8s	remaining: 2h 18m 42s
4:	learn: 0.5254697	total: 37.9s	remaining: 2h 11m 54s
5:	learn: 0.5188835	total: 42.3s	remaining: 2h 2m 36s
6:	learn: 0.5126898	total: 46.7s	remaining: 1h 56m 5s
7:	learn: 0.5077487	total: 51.9s	remaining: 1h 52m 36s
8:	learn: 0.5041109	total: 56.7s	remaining: 1h 49m 21s
9:	learn: 0.5013251	total: 1m 2s	remaining: 1h 48m 2s
10:	learn: 0.4994837	total: 1m 5s	remaining: 1h 43m 51s
11:	learn: 0.4973703	total: 1m 10s	remaining: 1h 41m 50s
12:	learn: 0.4952048	total: 1m 15s	remaining: 1h 40m 33s
13:	learn: 0.4937556	total: 1m 19s	remaining: 1h 37m 47s
14:	learn: 0.4928136	total: 1m 23s	remaining: 1h 36m 6s
15:	learn: 0.4909799	total: 1m 27s	remaining: 1h 34m 23s
16:	learn: 0.4889789	total: 1m 32s	remaining: 1h 33m 10s
17:	learn: 0.4877988	total

144:	learn: 0.3906881	total: 11m 28s	remaining: 1h 11m 37s
145:	learn: 0.3903098	total: 11m 33s	remaining: 1h 11m 35s
146:	learn: 0.3900387	total: 11m 38s	remaining: 1h 11m 28s
147:	learn: 0.3897749	total: 11m 42s	remaining: 1h 11m 20s
148:	learn: 0.3896377	total: 11m 46s	remaining: 1h 11m 13s
149:	learn: 0.3893163	total: 11m 51s	remaining: 1h 11m 7s
150:	learn: 0.3887670	total: 11m 56s	remaining: 1h 11m 7s
151:	learn: 0.3880478	total: 12m 2s	remaining: 1h 11m 10s
152:	learn: 0.3878023	total: 12m 8s	remaining: 1h 11m 13s
153:	learn: 0.3868642	total: 12m 14s	remaining: 1h 11m 11s
154:	learn: 0.3860022	total: 12m 18s	remaining: 1h 11m 3s
155:	learn: 0.3854422	total: 12m 22s	remaining: 1h 10m 56s
156:	learn: 0.3852792	total: 12m 28s	remaining: 1h 10m 56s
157:	learn: 0.3850968	total: 12m 33s	remaining: 1h 10m 53s
158:	learn: 0.3844135	total: 12m 38s	remaining: 1h 10m 48s
159:	learn: 0.3834390	total: 12m 43s	remaining: 1h 10m 44s
160:	learn: 0.3832233	total: 12m 48s	remaining: 1h 10m 45s
16

286:	learn: 0.3377640	total: 23m 18s	remaining: 1h 1m 56s
287:	learn: 0.3375718	total: 23m 22s	remaining: 1h 1m 51s
288:	learn: 0.3373156	total: 23m 27s	remaining: 1h 1m 46s
289:	learn: 0.3369240	total: 23m 31s	remaining: 1h 1m 40s
290:	learn: 0.3366881	total: 23m 36s	remaining: 1h 1m 35s
291:	learn: 0.3361777	total: 23m 40s	remaining: 1h 1m 28s
292:	learn: 0.3360931	total: 23m 45s	remaining: 1h 1m 22s
293:	learn: 0.3358168	total: 23m 49s	remaining: 1h 1m 15s
294:	learn: 0.3353789	total: 23m 54s	remaining: 1h 1m 12s
295:	learn: 0.3352538	total: 23m 58s	remaining: 1h 1m 4s
296:	learn: 0.3351260	total: 24m 3s	remaining: 1h 59s
297:	learn: 0.3345944	total: 24m 8s	remaining: 1h 54s
298:	learn: 0.3343058	total: 24m 12s	remaining: 1h 48s
299:	learn: 0.3342368	total: 24m 17s	remaining: 1h 44s
300:	learn: 0.3337659	total: 24m 22s	remaining: 1h 38s
301:	learn: 0.3335832	total: 24m 27s	remaining: 1h 33s
302:	learn: 0.3334584	total: 24m 31s	remaining: 1h 28s
303:	learn: 0.3332665	total: 24m 36s	r

434:	learn: 0.3012532	total: 34m 58s	remaining: 49m 26s
435:	learn: 0.3011437	total: 35m 3s	remaining: 49m 22s
436:	learn: 0.3010034	total: 35m 8s	remaining: 49m 17s
437:	learn: 0.3009079	total: 35m 12s	remaining: 49m 11s
438:	learn: 0.3007293	total: 35m 16s	remaining: 49m 6s
439:	learn: 0.3002762	total: 35m 21s	remaining: 49m 1s
440:	learn: 0.3001065	total: 35m 26s	remaining: 48m 56s
441:	learn: 0.2997641	total: 35m 30s	remaining: 48m 51s
442:	learn: 0.2995637	total: 35m 35s	remaining: 48m 45s
443:	learn: 0.2993871	total: 35m 41s	remaining: 48m 42s
444:	learn: 0.2992287	total: 35m 46s	remaining: 48m 38s
445:	learn: 0.2987420	total: 35m 51s	remaining: 48m 34s
446:	learn: 0.2985756	total: 35m 58s	remaining: 48m 31s
447:	learn: 0.2983776	total: 36m 3s	remaining: 48m 26s
448:	learn: 0.2982818	total: 36m 8s	remaining: 48m 22s
449:	learn: 0.2980549	total: 36m 12s	remaining: 48m 16s
450:	learn: 0.2979897	total: 36m 17s	remaining: 48m 12s
451:	learn: 0.2978849	total: 36m 22s	remaining: 48m 7s

582:	learn: 0.2737830	total: 47m 4s	remaining: 37m 42s
583:	learn: 0.2736663	total: 47m 10s	remaining: 37m 38s
584:	learn: 0.2735007	total: 47m 14s	remaining: 37m 33s
585:	learn: 0.2733484	total: 47m 19s	remaining: 37m 28s
586:	learn: 0.2732403	total: 47m 24s	remaining: 37m 23s
587:	learn: 0.2730926	total: 47m 29s	remaining: 37m 18s
588:	learn: 0.2730349	total: 47m 33s	remaining: 37m 13s
589:	learn: 0.2730071	total: 47m 38s	remaining: 37m 8s
590:	learn: 0.2728768	total: 47m 43s	remaining: 37m 4s
591:	learn: 0.2727628	total: 47m 48s	remaining: 36m 58s
592:	learn: 0.2726406	total: 47m 53s	remaining: 36m 54s
593:	learn: 0.2724471	total: 47m 58s	remaining: 36m 49s
594:	learn: 0.2722905	total: 48m 2s	remaining: 36m 44s
595:	learn: 0.2722290	total: 48m 7s	remaining: 36m 39s
596:	learn: 0.2721896	total: 48m 11s	remaining: 36m 34s
597:	learn: 0.2720674	total: 48m 16s	remaining: 36m 29s
598:	learn: 0.2718384	total: 48m 21s	remaining: 36m 24s
599:	learn: 0.2714145	total: 48m 26s	remaining: 36m 1

730:	learn: 0.2524521	total: 59m 8s	remaining: 25m 48s
731:	learn: 0.2523420	total: 59m 12s	remaining: 25m 43s
732:	learn: 0.2520193	total: 59m 17s	remaining: 25m 38s
733:	learn: 0.2518997	total: 59m 21s	remaining: 25m 33s
734:	learn: 0.2518372	total: 59m 26s	remaining: 25m 28s
735:	learn: 0.2517357	total: 59m 31s	remaining: 25m 23s
736:	learn: 0.2514272	total: 59m 35s	remaining: 25m 18s
737:	learn: 0.2512031	total: 59m 41s	remaining: 25m 13s
738:	learn: 0.2511023	total: 59m 45s	remaining: 25m 8s
739:	learn: 0.2509974	total: 59m 49s	remaining: 25m 3s
740:	learn: 0.2507964	total: 59m 55s	remaining: 24m 59s
741:	learn: 0.2506884	total: 1h 1s	remaining: 24m 54s
742:	learn: 0.2505792	total: 1h 5s	remaining: 24m 49s
743:	learn: 0.2503526	total: 1h 9s	remaining: 24m 44s
744:	learn: 0.2502738	total: 1h 13s	remaining: 24m 39s
745:	learn: 0.2500963	total: 1h 19s	remaining: 24m 34s
746:	learn: 0.2499376	total: 1h 23s	remaining: 24m 29s
747:	learn: 0.2498648	total: 1h 29s	remaining: 24m 25s
748:	

874:	learn: 0.2330220	total: 1h 10m 43s	remaining: 14m 8s
875:	learn: 0.2329567	total: 1h 10m 47s	remaining: 14m 3s
876:	learn: 0.2328047	total: 1h 10m 52s	remaining: 13m 58s
877:	learn: 0.2326886	total: 1h 10m 56s	remaining: 13m 53s
878:	learn: 0.2324390	total: 1h 11m 1s	remaining: 13m 49s
879:	learn: 0.2323147	total: 1h 11m 7s	remaining: 13m 44s
880:	learn: 0.2321581	total: 1h 11m 12s	remaining: 13m 39s
881:	learn: 0.2320660	total: 1h 11m 16s	remaining: 13m 34s
882:	learn: 0.2319193	total: 1h 11m 23s	remaining: 13m 30s
883:	learn: 0.2318037	total: 1h 11m 28s	remaining: 13m 25s
884:	learn: 0.2316522	total: 1h 11m 32s	remaining: 13m 20s
885:	learn: 0.2315814	total: 1h 11m 38s	remaining: 13m 15s
886:	learn: 0.2314681	total: 1h 11m 43s	remaining: 13m 10s
887:	learn: 0.2313601	total: 1h 11m 49s	remaining: 13m 6s
888:	learn: 0.2313394	total: 1h 11m 54s	remaining: 13m 1s
889:	learn: 0.2312698	total: 1h 11m 59s	remaining: 12m 56s
890:	learn: 0.2311130	total: 1h 12m 5s	remaining: 12m 51s
891:

1016:	learn: 0.2182980	total: 1h 22m 23s	remaining: 2m 40s
1017:	learn: 0.2182386	total: 1h 22m 28s	remaining: 2m 35s
1018:	learn: 0.2181551	total: 1h 22m 33s	remaining: 2m 30s
1019:	learn: 0.2181079	total: 1h 22m 38s	remaining: 2m 25s
1020:	learn: 0.2178854	total: 1h 22m 43s	remaining: 2m 20s
1021:	learn: 0.2178057	total: 1h 22m 47s	remaining: 2m 16s
1022:	learn: 0.2177106	total: 1h 22m 52s	remaining: 2m 11s
1023:	learn: 0.2176068	total: 1h 22m 56s	remaining: 2m 6s
1024:	learn: 0.2173991	total: 1h 23m 1s	remaining: 2m 1s
1025:	learn: 0.2172287	total: 1h 23m 6s	remaining: 1m 56s
1026:	learn: 0.2171401	total: 1h 23m 11s	remaining: 1m 51s
1027:	learn: 0.2170331	total: 1h 23m 16s	remaining: 1m 46s
1028:	learn: 0.2169798	total: 1h 23m 20s	remaining: 1m 42s
1029:	learn: 0.2168988	total: 1h 23m 24s	remaining: 1m 37s
1030:	learn: 0.2168189	total: 1h 23m 28s	remaining: 1m 32s
1031:	learn: 0.2166974	total: 1h 23m 33s	remaining: 1m 27s
1032:	learn: 0.2166166	total: 1h 23m 38s	remaining: 1m 22s
1

In [43]:
test = test.fillna(0)

In [44]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [45]:
test = test.drop("TailNum", axis = 1)

In [46]:
test = test.fillna(0)

In [47]:
test['DepTime'] = test['DepTime'] / 100

In [48]:
test['ArrTime'] = test['ArrTime'] / 100

In [49]:
test = test.drop("Id", axis = 1)

In [50]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [51]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [52]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,15,5,8.06,9.57,UA,1157,111.0,DFW,DEN,641,8.0,52.0,0.203920,0.301685
1,10,14,2,20.20,21.33,WN,1965,49.0,PHX,ONT,325,4.0,20.0,0.302108,0.418648
2,6,1,7,22.45,0.09,FL,910,67.0,ATL,PHF,508,6.0,11.0,0.297510,0.250599
3,12,31,3,16.38,17.53,OO,5732,66.0,GEG,SEA,224,3.0,6.0,0.314572,0.277956
4,10,23,4,15.44,23.58,AA,22,289.0,LAX,JFK,2475,12.0,13.0,0.289365,0.344834


In [53]:
proba = CatBoost.predict_proba(test)[:, 1]

In [54]:
submission = pd.read_csv("sample_submission.csv")

In [55]:
submission['target'] = proba

In [56]:
submission.to_csv("submission.csv", index = False, header = True)